# Tissue specific model loading

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [23]:
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Brain_liver_specific_plusthreeCompetitiveObesityMap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load Recon2.2Model with a simplified medium for the course
M = cobra.io.load_json_model('Brain_liver_specific_plusthreeCompetitive.json')

In [24]:
M.reactions.Brain_biomass_maintenance.lower_bound=0

In [25]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = M.problem.Objective(M.reactions.Brain_biomass_maintenance.flux_expression+
                                              M.reactions.get_by_id('Brain_3HLYTCL').flux_expression+
                                             M.reactions.get_by_id('Brain_5HLTDL').flux_expression,
    direction='max')
M.objective = quadratic_objective

In [26]:
M.solver='glpk'
sol = M.optimize()
sol.objective_value

3.6218423879711157

In [27]:
M1=M.copy()

In [28]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = M1.problem.Objective(M1.reactions.Brain_biomass_maintenance.flux_expression+
                                              M1.reactions.get_by_id('Brain_3HLYTCL').flux_expression+
                                             M1.reactions.get_by_id('Brain_5HLTDL').flux_expression,
    direction='max')
M1.objective = quadratic_objective

In [29]:
## Brain Phe
M1.reactions.Brain_EX_phe_L_e__csf.lower_bound=0
M1.reactions.Brain_EX_phe_L_e__csf.upper_bound=0
M1.reactions.Brain_PHETA1.lower_bound=0

In [30]:
## Liver Phe_L_c
M1.reactions.Liver_PHETA1.lower_bound=0
M1.reactions.Liver_RE2269E.upper_bound=0

In [31]:
## Brain Tyr
M1.reactions.Brain_EX_tyr_L_e__csf.lower_bound=0
M1.reactions.Brain_EX_tyr_L_e__csf.upper_bound=0
M1.reactions.Brain_EX_34dhphe_e__csf.lower_bound=0
M1.reactions.Brain_EX_34dhphe_e__csf.upper_bound=0
M1.reactions.Brain_TYRTA.lower_bound=0
M1.reactions.get_by_id('34dhpheTR_TOBD').upper_bound=0
M1.reactions.Brain_TYRDOPO.upper_bound=0

In [32]:
## Brain Trp
M1.reactions.Brain_EX_trp_L_e__csf.lower_bound=0
M1.reactions.Brain_EX_trp_L_e__csf.upper_bound=0

In [33]:
## Reaction has upper_bound lower than 0, which will be changed to 0
M1.reactions.Liver_EX_ala_L_e.upper_bound=0
M1.reactions.Brain_EX_o2_e__csf.upper_bound=0
M1.reactions.Brain_EX_glc_D_e__csf.upper_bound=0

In [34]:
## Reaction has lower_bound higher than 0, which will be changed to 0
for rxn in M1.reactions:
    if M1.reactions.get_by_id(rxn.id).lower_bound>0:
        M1.reactions.get_by_id(rxn.id).lower_bound=0

In [35]:
M1.solver='glpk'
sol = M1.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.4262626210295204

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Liver_10FTHFtm': -0.019361, …

In [36]:
## Model setting
M1.reactions.Liver_EX_tyr_L_e.lower_bound=-0.04
M1.reactions.Liver_EX_trp_L_e.lower_bound=-0.1
M1.reactions.Liver_EX_phe_L_e.lower_bound=-0.2
M1.reactions.Liver_EX_phe_L_e.upper_bound=-0.2
M1.reactions.Liver_biomass_reaction.lower_bound=0.05
M1.reactions.Liver_biomass_reaction.upper_bound=0.05
M1.reactions.Brain_biomass_maintenance.upper_bound=0.1
M1.reactions.Brain_5HLTDL.lower_bound=0.095
M1.reactions.Ti_To.upper_bound=0.195
M1.reactions.Liver_PHETA1.upper_bound=0
M1.reactions.Liver_TYRTA.lower_bound=0
M1.reactions.Liver_RE2269E.lower_bound=0
M1.reactions.Liver_HMR_7628.upper_bound=0
M1.reactions.Liver_r1455.upper_bound=0
M1.reactions.Liver_biomass_maintenance.upper_bound=0
M1.reactions.Brain_Phe_TH.upper_bound=0

In [37]:
M1.reactions.Liver_EX_glc_D_e.lower_bound=-1

In [38]:
M1.solver='glpk'
sol = M1.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.25175539999999996

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Liver_10FTHFtm': -0.019785, …

In [39]:
## Essential needs
M1.reactions.Brain_biomass_maintenance.lower_bound=0.1
M1.reactions.Brain_3HLYTCL.lower_bound=0.0567
M1.reactions.Brain_5HLTDL.lower_bound=0.095

In [40]:
exchanges = [ rxn for rxn in M1.reactions if rxn.products == [] and 'EX_' in rxn.id ]
for rxn in exchanges:
    rxn.upper_bound = 999999
M1.reactions.Liver_EX_phe_L_e.upper_bound=-0.2

In [41]:
M1.reactions.Liver_LPS.lower_bound=-1000

In [42]:
M1.objective='Liver_EX_tag_hs_e'
M1.solver='glpk'
sol = M1.optimize()
sol.objective_value

0.7408731888814316

In [43]:
fbasol =pfba(M1)
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Liver_10FTHFtm': -0.0, 'Live…

In [44]:
# To simulate the case for PKU patient (low Phe diet and high glucose)
model=M1.copy()
# Kncoking gene '5053' to simulate the PKU patients
model.genes.get_by_id('5053').knock_out()
model.reactions.Liver_PHETA1.upper_bound=0.1
model.reactions.Liver_EX_phe_L_e.upper_bound=-0.04
model.reactions.Liver_EX_phe_L_e.lower_bound=-0.04
model.reactions.Liver_EX_tyr_L_e.lower_bound=-0.2
## The Phe-restricted diet contains more carbohydrate content, We assumed more glucose.
model.reactions.Liver_EX_glc_D_e.lower_bound=-2
sol = model.optimize()
sol.objective_value

0.8151504656544816

In [45]:
fbasol = pfba(model)
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Liver_10FTHFtm': -0.0, 'Live…

In [46]:
## Normal person uptake more glucose
model2=M1.copy()
model2.reactions.Liver_EX_glc_D_e.lower_bound=-2
sol = model2.optimize()
sol.objective_value

0.8151504656544816

In [47]:
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Liver_10FTHFtm': 2.576828, '…

In [48]:
## PKU patient uptake same glucose as normal person
model3=M1.copy()
# Kncoking gene '5053.1' to simulate the PKU patients
model3.genes.get_by_id('5053').knock_out()
model3.reactions.Liver_PHETA1.upper_bound=0.1
model3.reactions.Liver_EX_phe_L_e.upper_bound=-0.04
model3.reactions.Liver_EX_phe_L_e.lower_bound=-0.04
model3.reactions.Liver_EX_tyr_L_e.lower_bound=-0.2
## The Phe-restricted diet contains more carbohydrate content, We assumed more glucose.
model3.reactions.Liver_EX_glc_D_e.lower_bound=-1
sol = model3.optimize()
sol.objective_value

0.740873191586966

In [49]:
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Liver_10FTHFtm': 1.10108, 'L…